Libraries for Project

In [1]:
import numpy as np
import pandas as pd # type: ignore

Load Data

In [6]:
Org_data = pd.read_csv("/home/malaka/Projects/CV_Projects/Datasets/Org_data.csv")
Org_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [7]:
print(Org_data.isnull().sum())

# Example: Drop rows with missing latitude or longitude (important for geospatial analysis)
df = Org_data.dropna(subset=['LAT', 'LON'])

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           147473
Vict Age               0
Vict Sex          140607
Vict Descent      140619
Premis Cd             16
Premis Desc          582
Weapon Used Cd    661838
Weapon Desc       661838
Status                 1
Status Desc            0
Crm Cd 1              11
Crm Cd 2          918239
Crm Cd 3          984577
Crm Cd 4          986813
LOCATION               0
Cross Street      834910
LAT                    0
LON                    0
dtype: int64


In [8]:
df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

/tmp/ipykernel_7811/2630310296.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])
/tmp/ipykernel_7811/2630310296.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])


In [9]:
df['Vict Age'] = pd.to_numeric(df['Vict Age'], errors='coerce')
df = df.drop(['DR_NO', 'Cross Street'], axis=1)

In [10]:
# Extract year and month from 'DATE OCC'
df['Year'] = df['DATE OCC'].dt.year
df['Month'] = df['DATE OCC'].dt.month

In [11]:
df['Vict Sex'] = df['Vict Sex'].map({'M': 0, 'F': 1, 'X': 2})  # 'X' for unknown

In [12]:
print(df.duplicated().sum())
df = df.drop_duplicates()


56224


In [13]:
print(df.describe())
print(df['Crm Cd Desc'].value_counts())


                           Date Rptd                       DATE OCC  \
count                         930649                         930649   
mean   2022-06-05 12:42:37.188369920  2022-05-26 02:14:51.241273856   
min              2020-01-01 00:00:00            2020-01-01 00:00:00   
25%              2021-04-11 00:00:00            2021-03-27 00:00:00   
50%              2022-07-17 00:00:00            2022-07-05 00:00:00   
75%              2023-07-18 00:00:00            2023-07-07 00:00:00   
max              2024-11-24 00:00:00            2024-11-24 00:00:00   
std                              NaN                            NaN   

            TIME OCC           AREA    Rpt Dist No       Part 1-2  \
count  930649.000000  930649.000000  930649.000000  930649.000000   
mean     1339.564049      10.233365    1069.757157       1.399806   
min         1.000000       1.000000     101.000000       1.000000   
25%       900.000000       5.000000     555.000000       1.000000   
50%      1420.0